In [11]:
import pandas as pd
import numpy as np
train = pd.read_csv("./train.csv")
test = pd.read_csv("./test.csv")

In [12]:
def kesson_table(df): 
        null_val = df.isnull().sum()
        percent = 100 * df.isnull().sum()/len(df)
        kesson_table = pd.concat([null_val, percent], axis=1)
        kesson_table_ren_columns = kesson_table.rename(
        columns = {0 : '欠損数', 1 : '%'})
        return kesson_table_ren_columns
kesson_table(train)

,欠損数,%
PassengerId,0,0.000000
Survived,0,0.000000
Pclass,0,0.000000
Name,0,0.000000
Sex,0,0.000000
Age,177,19.865320
SibSp,0,0.000000
Parch,0,0.000000
Ticket,0,0.000000
Fare,0,0.000000


In [13]:
kesson_table(test)

,欠損数,%
PassengerId,0,0.000000
Pclass,0,0.000000
Name,0,0.000000
Sex,0,0.000000
Age,86,20.574163
SibSp,0,0.000000
Parch,0,0.000000
Ticket,0,0.000000
Fare,1,0.239234
Cabin,327,78.229665


In [14]:
train.dtypes

PassengerId      int64
Survived         int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
dtype: object

In [15]:
train["Age"] = train["Age"].fillna(train["Age"].median())
train["Embarked"] = train["Embarked"].fillna("S")
kesson_table(train)

,欠損数,%
PassengerId,0,0.000000
Survived,0,0.000000
Pclass,0,0.000000
Name,0,0.000000
Sex,0,0.000000
Age,0,0.000000
SibSp,0,0.000000
Parch,0,0.000000
Ticket,0,0.000000
Fare,0,0.000000


In [28]:
for i in train.columns:
    if train[i].dtypes == 'object':
        train[i] = train[i].factorize()[0]

In [30]:
test["Age"] = test["Age"].fillna(test["Age"].median())
for i in test.columns[test.dtypes == 'object']:
    test[i] = test[i].factorize()[0]
test.Fare[152] = test.Fare.median()
test.head(10)

C:\Users\owner\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,0,0,34.5,0,0,0,7.8292,-1,0
1,893,3,1,1,47.0,1,0,1,7.0000,-1,1
2,894,2,2,0,62.0,0,0,2,9.6875,-1,0
3,895,3,3,0,27.0,0,0,3,8.6625,-1,1
4,896,3,4,1,22.0,1,1,4,12.2875,-1,1
5,897,3,5,0,14.0,0,0,5,9.2250,-1,1
6,898,3,6,1,30.0,0,0,6,7.6292,-1,0
7,899,2,7,0,26.0,1,1,7,29.0000,-1,1
8,900,3,8,1,18.0,0,0,8,7.2292,-1,2
9,901,3,9,0,21.0,2,0,9,24.1500,-1,1


In [31]:
# scikit-learnのインポートをします
from sklearn.ensemble import GradientBoostingClassifier
target = train["Survived"].values
train['FamilySize'] = train['Parch'] + train['SibSp'] + 1
test['FamilySize'] = test['Parch'] + test['SibSp'] + 1
feature = train[["Pclass", "Sex", "Age", "Fare", "FamilySize","Embarked"]].values
my_tree = GradientBoostingClassifier(n_estimators=55, random_state=0,min_samples_leaf=2)
my_tree = my_tree.fit(feature, target)
test_feature = test[["Pclass", "Sex", "Age", "Fare", "FamilySize","Embarked"]].values
my_prediction = my_tree.predict(test_feature)

In [33]:
PassengerId = np.array(test["PassengerId"]).astype(int)
my_solution = pd.DataFrame(my_prediction, PassengerId, columns = ["Survived"])
my_solution.to_csv("my_tree_GB.csv", index_label = ["PassengerId"])